# Elixirの関数とモジュール

Elixirは関数型言語で、関数は基本的な型の一つでもある。


- Elixirの関数は2種類
  - コードのどこにでも書ける無名関数(anonymous function)
  - モジュール(module)に属する必要がある名前付き関数(named function)
  

### 無名関数

モジュールに属さない関数。
変数に束縛することで使い回すことができる。

- 無名関数の定義方法
  - 基本
    - `fn (`args`) -> `return value` end`
  - 引数の括弧は省略可能
    - `fn `args` -> `return value` end`
- 無名関数は, 内部的には定義時に返る`#Function<43.97283095/2 in :erl_eval.expr/5>`のようなハッシュで識別されている


- 無名関数を変数に束縛できる
- 無名関数の呼び出し時は, `add.(1, 2)`のように`.()`とカッコの前にピリオドが必要
  - 名前付き関数呼び出しの`()`が省略できる仕様との間で曖昧さを回避するため
  - `IO.inspect "hello"`は`IO` moduleの名前付き関数`puts`を呼び出しているので括弧を省略できている
  - 丁寧に書くと`IO.inspect("hello")`

In [ ]:
fn x, y -> x + y end

In [ ]:
add = fn (a, b) -> a + b end
mul = fn a, b -> a * b end

IO.inspect add.(1, 2)
IO.inspect mul.(3, 2)

引数名を省略する記法もある
- `&()`で囲み, `&1`, `&2`, ... で第1引数から順に参照する
- 後述の高階関数に単純な関数を与える際などに使うことがある

In [ ]:
# fn a, b -> a + b end と同等
&(&1 + &2)

# fn f, arg -> f.(arg) end と同等
&(&1.(&2))

式を複数持つ無名関数を定義することもできる。

関数内の最後の式の値が返り値になる。

In [ ]:
verbose_add = fn (a, b) ->
  IO.puts a
  IO.puts b
  a + b
end

verbose_add.(2, 3)

おまけクイズ
無名関数を変数に束縛せずに呼び出すことも可能。どうすればいいだろうか? 他言語では即時関数と呼ばれている場合がある。

In [ ]:
# Q. 変数に束縛せず2つの値を引数にとってその和を返す関数を定義し, かつその場で無名関数を1と2に適用せよ。(返り値の期待値は3)


## 高階関数

引数に関数を取ったり、関数を返り値としたりする関数。

In [ ]:
# 関数 f を受け取って, 2つの引数を f に適用する関数を返す関数
my_apply2 = fn f -> 
  fn x, y -> f.(x, y) end 
end

In [ ]:
# 和と積を計算する関数
my_add = my_apply2.(&(&1 + &2))
my_mul = my_apply2.(&(&1 * &2))

In [ ]:
IO.inspect my_add.(6, 7)
IO.inspect my_mul.(6, 7)

### 練習問題

第1引数，第2引数に数字，第3引数に引数を2つとる関数をとり, 第1引数と第2引数をそれぞれ第3引数の関数へ渡して実行する関数を作ってみよう。

（fnを使う記法と&を使う記法と両方で作ってみよう）

In [ ]:
func = fn -> end # implement me!

add = &(&1 + &2)
func.(1, 2, add) == 3

In [ ]:
# 左辺で関数定義と関数適用を完結させてください
== 3

### モジュールと名前付き関数

Elixirでは関連する関数をグループ化してモジュールとして管理する．

処理を行う対象となるデータごとにモジュールを分割する事が多い。

- e.g.) 文字列を処理する`String`モジュールなど

モジュールは階層化することができる。


モジュール内で定義した関数は名前付き関数となる。
- `def`でモジュール外から呼び出せるpublic関数を定義
- `defp`でモジュール内からしか呼び出せないprovate関数を定義


In [ ]:
defmodule MyMath do
  def add(x, y) do
    x + y
  end
  
  def multiple(x, y) do
    x * y
  end
  
  def get_sum_and_products(x, y) do
    show_args(x, y)
    {add(x, y), multiple(x, y)}
  end
  
  defp show_args(x, y) do
    IO.inspect "Called with x: #{x}, y: #{y}"
  end
end

In [ ]:
IO.inspect MyMath.add(1, 2)
IO.inspect MyMath.multiple(2, 3)

In [ ]:
MyMath.get_sum_and_products(2, 3)

In [ ]:
MyMath.show_args(2, 3)
# => %UndefinedFunctionError{arity: 2, function: :show_args, message: nil, module: MyMath, reason: nil}

In [ ]:
# 名前付き関数適用時の括弧は省略可能
MyMath.add 1, 2

高階関数に名前付き関数を渡す時は、`&ModuleName.function_name/0`や`&function_name/0`のように、`/0`でarity(引数の数)を指定する。

Elixirではarityが違う関数は異なるものとして扱われる。

In [ ]:
defmodule Vegitable do
  def apply_get_name(get_name_func) do
    get_name_func.()
  end

  def apply_get_name(get_name_func, adjective) do
    get_name_func.(adjective)
  end
end

defmodule Vegitable.Tomato do
  def get_name() do
    "tomato"
  end
  
  def get_name(adjective) do
    "#{adjective} tomato"
  end
end

In [ ]:
Vegitable.apply_get_name(&Vegitable.Tomato.get_name/0)

In [ ]:
Vegitable.apply_get_name(&Vegitable.Tomato.get_name/1, "sweet")

In [ ]:
Vegitable.apply_get_name(&Vegitable.Tomato.get_name/0, "sweet")

### 関数の多重定義

Elixirでは、同じ名前・arityの関数を複数定義することができる。

このとき、引数のパターンマッチにより適用を分岐することができる。

無名関数と名前付き関数で定義方法が若干異なる。

In [ ]:
overloaded_func = fn
  {:ok, value}  -> IO.puts "OK #{value}"
  {:error, msg} -> IO.puts "Error #{msg}"
  _             -> IO.puts "not match"
end

overloaded_func.({:ok, "good"})
overloaded_func.({:error, "bad"})
overloaded_func.(:ok)

In [ ]:
defmodule OverloadExample do
  def ensure_success({:ok, _} = result) do
    IO.puts "Succeeded"
    result
  end
  
  def ensure_success({:error, message} = result) do
    IO.puts "Failed (#{message})"
    result
  end
  
  def ensure_success(_) do
    IO.puts "Something wrong"
  end
end

OverloadExample.ensure_success({:ok, 42})
OverloadExample.ensure_success({:error, "No answer is found"})
OverloadExample.ensure_success(:not_considered)

### 関数の仕様(スペック)

Elixirは動的型付け言語なので、ランタイム時の型チェックは難しい。

しかし、型自体は存在しており、関数のスペック(引数や返り値の型)を表現することは可能。

スペックを定義することで、静的解析ツールを用いてコンパイル時にチェックさせることが可能。


In [ ]:
defmodule SpecExample do
  @spec add(number, number) :: number
  def add(x, y) do
    x + y
  end
end